In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. O primeiro passo da exploração de dados é obter informações gerais, deve-se importar o arquivo “covid_19_clean_complete.csv” e obter as seguintes informações.

In [3]:
covid19 = pd.read_csv('/kaggle/input/corona-virus-report/covid_19_clean_complete.csv')
covid19.head()

In [37]:
# a. Quantidade de linhas e colunas
len(covid19)   # linhas
len(covid19.columns)   # colunas

In [38]:
# b. Quais são as colunas
covid19.columns.tolist()

In [39]:
# c. O tipo de dado de cada coluna
covid19.dtypes

In [40]:
# d. A coluna de datas deve ser transformada de “Object” para “datetime64”, utilizando o comando “pd.todatetime( )”
covid19.Date = covid19.Date.astype('datetime64')

In [41]:
# e. Informações estatísticas sobre o banco de dados
covid19.describe()

In [42]:
# f. Se necessário, transformar uma coluna pertinente para formato de data

# g. Quais colunas apresentam NaN
covid19.columns[covid19.isnull().any()].tolist()

# 2. Crie um dataframe que contenha apenas as 5 províncias da China que mais registraram casos.

In [4]:
covid19 = covid19.rename(columns={'Country/Region': 'Country', 'Province/State': 'Province'})
covid19_china = covid19.loc[covid19.Country == 'China']
covid19_china.head()
#covid19_china.sort_values(by='Confirmed', ascending=False)


In [98]:
#a. Descubra qual o nome das províncias chinesas contidas no dataset
covid19_china.Province.unique().tolist()

In [120]:
#b. Retire do banco de dados apenas as informações dessas províncias
covid19_china.groupby('Province').Confirmed.agg([len, sum]).sort_values(by='sum', ascending=False)

In [141]:
#c. Tome apenas as informações das features: “Confirmed”, “Active”, “Deaths”, “Recovered”
covid19_china = covid19_china.loc[:, ['Province', 'Confirmed', 'Active', 'Deaths', 'Recovered']]
covid19_china

In [171]:
#d. Agrupe o novo dataframe a partir dessas províncias. Qual a função de agrupamento que deve ser usada?
covid19_china = covid19_china.groupby('Province')


In [173]:
#e. Produza um dataframe contendo as 5 regiões com o maior número de casos confirmados
covid19_china.sum().sort_values(by='Confirmed', ascending=False).head(5)

# 3. A coluna “Province/State” contém muitos valores faltando. Para descartá-la perdendo o mínimo de informação, inclua na coluna “Country/Region” o nome das regiões junto ao nome dos países em que existem mais de uma província registrada no banco de dados, utilizando o método “.apply”. Como por exemplo, juntar o nome China com a região pertinente: “China_Hubei”, “China_Guangdong”.


In [196]:
#a. Elabore uma função que tenha uma linha do banco de dados como argumento e, se a coluna 
#“Province/State” não for valor faltante, concatenar seu valor com coluna “Country/Region”
    # i. Utilize o método “pandas.notna” para averiguar de “Province/State” é valor faltante

def concat_columns(row):
    if pd.notna(row.Province):
        row.Country = row.Country + '_' + row.Province
    return row



In [207]:
# b. Faça uma cópia do banco de dados para poder retirar informações sem perder o banco de dados original
# c. Aplique, por meio do método “apply”, a função criada no item a
covid19_cp = covid19.apply(concat_columns, axis='columns')

In [208]:
# d. Exclua a coluna “Province/State” do novo dataframe
covid19_cp = covid19_cp.drop('Province', 1)

In [211]:
covid19_cp[pd.notna(covid19.Province)]

# 4. Importe o arquivo "world meter data.csv" e, a partir de seus dados de população e continentes, faça um ranking de maior número de mortes por milhão de habitantes entre os continentes

In [5]:
#a. Importar o banco de dados "world meter data.csv"
worldometer_data = pd.read_csv('/kaggle/input/corona-virus-report/worldometer_data.csv')
worldometer_data = worldometer_data.rename(columns={'Country/Region': 'Country'})
worldometer_data.head()

In [6]:
#b. Tome apenas as informações de população, país e continentes deste banco de dados
worldometer_data = worldometer_data.loc[:, ['Country', 'Continent', 'Population',]]
worldometer_data = worldometer_data.set_index('Country')
worldometer_data.head()

In [7]:
#c. Agrupe o “covid_19_clean_complete” por países. Deve-se usar o dataframe do item “c)” ou o original?
covid19 = covid19.groupby('Country').sum()


In [8]:
#d. Mescle os dois dataframes, a partir dos países, contendo apenas as informações de mortes, população e continente
merged_df = worldometer_data.join(covid19)
merged_df = merged_df.drop(columns=['Lat', 'Long','Confirmed', 'Recovered', 'Active'])

In [9]:
merged_df.head()

In [10]:
#e. Agrupe o novo dataset agora pelo continente
merged_df = merged_df.groupby('Continent').sum()

In [19]:
#f. Crie uma nova coluna contendo o número de mortes por milhão dentro dos continentes (Utilize a fórmula: (mortes/população)*10^6)
merged_df['Deaths_Normalized']=(merged_df.Deaths/merged_df.Population)*1E6

In [23]:
#g. Faça um ranking do número de mortes por milhão de habitantes entre os continentes
merged_df.sort_values(by = 'Deaths_Normalized', ascending=False)